In [1]:
import spacy
import re
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from nav_pii_anon.spacy.spacy_model import SpacyModel
from nav_pii_anon.regex_engine import fnr
from nav_pii_anon.regex_engine.tlfnr import RegexTlfNr
from nav_pii_anon.regex_engine.date_time import RegexDateTime

## Using the Entity Ruler object from SpaCy

### Assigning multiple custom entities

In [3]:
TRAIN_DATA = [
    (
        "Horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, 'TLF')]},
    ),
    ("Do they bite?", {"entities": []}),
    (
        "horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, 'TLF')]},
    ),
    ("horses pretend to care about your feelings", {"entities": [(0, 6, 'TLF')]}),
    (
        "they pretend to care about your feelings, those horses",
        {"entities": [(48, 54, 'TLF')]},
    ),
    ("horses?", {"entities": [(0, 6, 'TLF')]}),
]

In [4]:
nlp = SpacyModel()
nlp.add_patterns()

#nlp.train(TRAIN_DATA, labels=['TLF'])
nlp.disable_ner()

In [5]:
tx="Jeg er johanne Nymark og mitt fødselsnummer er 15044216652 og tlf er +47 99 11 92 34 . Jeg kommer innom for å \
snakke om min 100 prosent stilling, og bergen. 13.09.1994. 4912 5900 2422 0342"
type(tx)

str

In [6]:
nlp.predict(tx)

[['15044216652', 'FNR', 8, 9, 'NA'], ['+47 99 11 92 34', 'TLF', 12, 17, 'NA'], ['100 prosent', 'AMOUNT', 26, 28, 'NA'], ['4912 5900 2422 0342', 'CREDIT_CARD', 34, 38, 'NA']]


In [6]:
tx[35:39]

'lsnu'

In [7]:
nlp.confusion_matrix(TRAIN_DATA)

ner (are,) regex []
Correct regex  []
No overlap
first runthrough [are]
[are]
ner (Do,) regex []
Correct regex  []
No overlap
first runthrough [Do]
[Do]
ner (are,) regex []
Correct regex  []
No overlap
first runthrough [are]
[are]
ner () regex []
Correct regex  []
[]
ner (those horses,) regex []
Correct regex  []
No overlap
first runthrough [those horses]
[those horses]
ner () regex []
Correct regex  []
[]
ner (are,) regex []
Correct regex  []
No overlap
first runthrough [are]
[are]
ner (Do,) regex []
Correct regex  []
No overlap
first runthrough [Do]
[Do]
ner (are,) regex []
Correct regex  []
No overlap
first runthrough [are]
[are]
ner () regex []
Correct regex  []
[]
ner (those horses,) regex []
Correct regex  []
No overlap
first runthrough [those horses]
[those horses]
ner () regex []
Correct regex  []
[]


,Is Positive,Is Negative
Predicted Positive,0,4
Predicted Negative,5,38


In [8]:
nlp.display_predictions(tx)

ner (johanne Nymark, bergen) regex [15044216652, 4912 5900 2422 0342, +47 99 11 92 34, 92 34, 0342, 100 prosent]
Correct regex  [15044216652, 4912 5900 2422 0342, +47 99 11 92 34, 100 prosent]
No overlap
first runthrough [johanne Nymark]
No overlap
first runthrough [johanne Nymark, bergen]
No overlap
second runthrough [johanne Nymark, bergen, 15044216652]
No overlap
second runthrough [johanne Nymark, bergen, 15044216652, 4912 5900 2422 0342]
No overlap
second runthrough [johanne Nymark, bergen, 15044216652, 4912 5900 2422 0342, +47 99 11 92 34]
No overlap
second runthrough [johanne Nymark, bergen, 15044216652, 4912 5900 2422 0342, +47 99 11 92 34, 100 prosent]
[johanne Nymark, bergen, 15044216652, 4912 5900 2422 0342, +47 99 11 92 34, 100 prosent]
